In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

In [69]:
class MemNN(nn.Module):
    def __init__(self, vocab_size, embedding_size, story_max_len, query_max_len):
        super(MemNN, self).__init__()
        
        self.A = nn.Embedding(vocab_size, embedding_size)
        self.B = nn.Embedding(vocab_size, embedding_size)
        self.C = nn.Embedding(vocab_size, query_max_len)
        
        self.fc_a = nn.Linear(embedding_size, vocab_size)
        
    def forward(self, story, query):
        
        story_a = self.A(story)
        query = self.B(query)
        story_c = self.C(story)
        
        p = F.softmax(torch.mm(query, story_a.permute(1, 0)))
        #should be shape (story_len, query_len)
                      
        #print(p)
        
        story_c = story_c.permute(1, 0)
        
        o = p.mul(story_c)
        
        print(o.size())
        print(query.size())
        a = o.add(query)
        
        print(a)
        
        return story_a, query, story_c

In [70]:
story = Variable(torch.LongTensor([1,2,3,4,5,6,7,8,9,0]))
query = Variable(torch.LongTensor([1,2,3,4,5]))

memnn = MemNN(10, 100, 10, 5)

In [71]:
story_a, query, story_c = memnn(story, query)

torch.Size([5, 10])
torch.Size([5, 100])


RuntimeError: inconsistent tensor size, expected r_ [5 x 10], t [5 x 10] and src [5 x 100] to have the same number of elements, but got 50, 50 and 500 elements respectively at /opt/conda/conda-bld/pytorch_1503965122592/work/torch/lib/TH/generic/THTensorMath.c:887